# Load libraries

Import the required libraries.

In [1]:
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib.pyplot as plt
from matplotlib import colors

import tensorflow as tf
import numpy as np


from tqdm import tqdm
from typing import Callable, Tuple

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from tensorflow.keras import backend as K
from tensorflow.python.keras.utils.losses_utils import reduce_weighted_loss

import kagglehub

2024-10-09 17:01:45.611512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 17:01:45.688596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 17:01:45.710244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 17:01:45.858842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 17:01:47.113633: W tensorflow/compiler/tf2

In [13]:
# Download latest version
path = kagglehub.dataset_download("fantineh/next-day-wildfire-spread")

print("Path to dataset files:", path)

Path to dataset files: /home/dtlitster/.cache/kagglehub/datasets/fantineh/next-day-wildfire-spread/versions/2


# Prepare data

In [3]:
from data import new_data_utils as ndu

In [5]:
BATCH_SIZE = 128
path = "/home/dtlitster/.cache/kagglehub/datasets/fantineh/next-day-wildfire-spread/versions/2"
train_dataset = ndu.get_dataset(path + "*", 
    data_size=64, sample_size=32, batch_size=BATCH_SIZE,
    num_in_channels=12, compression_type=None, clip_and_normalize=True,
    clip_and_rescale=False, random_crop=True, center_crop=False)
# 
# validation_dataset = get_dataset('/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_eval*', 
#     data_size=64, sample_size=32, batch_size=BATCH_SIZE,
#     num_in_channels=12, compression_type=None, clip_and_normalize=True,
#     clip_and_rescale=False, random_crop=True, center_crop=False)
# 
# test_dataset = get_dataset('/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_test*',
#     data_size=64, sample_size=64, batch_size=BATCH_SIZE,
#     num_in_channels=12, compression_type=None, clip_and_normalize=True,
#     clip_and_rescale=False, random_crop=False, center_crop=False)

In [6]:
print(train_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 12), dtype=tf.float32, name=None), TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name=None))>


# Visualize data
We will check content of the dataset by plotting them

Let's plot the data!

First we define the names for each of our variables.

In [7]:
TITLES = [
  'Elevation',
  'Wind\ndirection',
  'Wind\nvelocity',
  'Min\ntemp',
  'Max\ntemp',
  'Humidity',
  'Precip',
  'Drought',
  'Vegetation',
  'Population\ndensity',
  'Energy\nrelease\ncomponent',
  'Previous\nfire\nmask',
  'Fire\nmask'
]

Define some helper variables for the plot. 

In [8]:
def plot_samples_from_dataset(dataset: tf.data.Dataset, n_rows: int):
    """
    Plot 'n_rows' rows of samples from dataset.
    
    Args:
        dataset (Dataset): Dataset from which to plot samples.
        n_rows (int): Number of rows to plot.
    """
    global TITLES
    
    # Get batch
    inputs, labels = None, None
    for elem in dataset:
        inputs, labels = elem
        break
    
    fig = plt.figure(figsize=(15,6.5))

    # Variables for controllong the color map for the fire masks
    CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
    BOUNDS = [-1, -0.1, 0.001, 1]
    NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
    # Number of data variables
    n_features = 12
    for i in range(n_rows):
        for j in range(n_features + 1):
            plt.subplot(n_rows, n_features + 1, i * (n_features + 1) + j + 1)
            if i == 0:
                plt.title(TITLES[j], fontsize=13)
            if j < n_features - 1:
                plt.imshow(inputs[i, :, :, j], cmap='viridis')
            if j == n_features - 1:
                plt.imshow(inputs[i, :, :, -1], cmap=CMAP, norm=NORM)
            if j == n_features:
                plt.imshow(labels[i, :, :, 0], cmap=CMAP, norm=NORM) 
            plt.axis('off')
    plt.tight_layout()
    

In [9]:
plot_samples_from_dataset(train_dataset, 5)

2024-10-09 17:03:59.045333: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: FAILED_PRECONDITION: /home/dtlitster/.cache/kagglehub/datasets/fantineh/next-day-wildfire-spread/versions/2; Is a directory


FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /home/dtlitster/.cache/kagglehub/datasets/fantineh/next-day-wildfire-spread/versions/2; Is a directory [Op:IteratorGetNext] name: 

# Building testing

## Metrics
Let's define metric calculation functions

In [ ]:
def IoU_metric(real_mask: tf.Tensor, predicted_mask: tf.Tensor) -> float:
    """
    Calculation of intersection over union metric.
    
    Args:
        real_mask (Tensor): Ground-truth mask
        predicted_mask (Tensor): Mask predicted by model
    Returns:
        (float): IoU metric value
    """
    real_mask = tf.where(real_mask < 0, 0, real_mask)
    
    intersection = np.logical_and(real_mask, predicted_mask)
    union = np.logical_or(real_mask, predicted_mask)
    
    if np.sum(union) == 0:
        return 1
    return np.sum(intersection) / np.sum(union)

def recall_metric(real_mask: tf.Tensor, predicted_mask: tf.Tensor) -> float:
    """
    Calculation of recall metric.
    
    Args:
        real_mask (Tensor): Ground-truth mask
        predicted_mask (Tensor): Mask predicted by model
    Returns:
        (float): recall metric value
    """
    real_mask = tf.where(real_mask < 0, 0, real_mask)
    
    true_positives = np.sum(np.logical_and(real_mask, predicted_mask))
    actual_positives = np.sum(real_mask)
    if actual_positives == 0:
        return 1
    
    return true_positives / actual_positives

def precision_metric(real_mask: tf.Tensor, predicted_mask: tf.Tensor) -> float:
    """
    Calculation of precision metric.
    
    Args:
        real_mask (Tensor): Ground-truth mask
        predicted_mask (Tensor): Mask predicted by model
    Returns:
        (float): precision metric value
    """
    real_mask = tf.where(real_mask < 0, 0, real_mask)
    
    true_positives = np.sum(np.logical_and(real_mask, predicted_mask))
    predicted_positives = np.sum(predicted_mask)
    if predicted_positives == 0:
        return 1
    
    return true_positives / predicted_positives

## Loss functions

In [ ]:
def dice_coef(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """
    Dice loss function calculator.
    
    Args:
        y_true (Tensor): 
        y_pred (Tensor):
    Returns:
        (Tensor): Dice loss for each element of a batch.
    """
    smooth = 1e-6
    y_true_f = K.reshape(y_true, (BATCH_SIZE, -1))
    y_pred_f = K.reshape(y_pred, (BATCH_SIZE, -1))
    intersection = K.sum(y_true_f * y_pred_f, axis=1)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f, axis=1) + K.sum(y_pred_f, axis=1) + smooth)

def weighted_bincrossentropy(true: tf.Tensor, pred: tf.Tensor, weight_zero: float = 0.01, weight_one: float = 1) -> float:
    """
    Calculates weighted binary cross entropy. The weights are fixed.
        
    This can be useful for unbalanced catagories.
    
    Adjust the weights here depending on what is required.
    
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
        will be penalize 10 times as much as false negatives.
    
    Args:
        true (Tensor): Ground-truth values
        pred (Tensor): Predited values
        weight_zero (float): Weight of class 0 (no-fire)
        weight_one (float): Weight of class 1 (fire)
        
    """
  
    # calculate the binary cross entropy
    bin_crossentropy = K.binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy 
    
    return K.mean(weighted_bin_crossentropy, axis=1)

def bce_dice_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    """
    BCE loss function calculator.
    
    Args:
        y_true (Tensor): 
        y_pred (Tensor):
    Returns:
        (Tensor): Mean BCE Dice loss over a batch.
    """
    
    y_true_f = K.reshape(y_true, (BATCH_SIZE, -1))
    y_pred_f = K.reshape(y_pred, (BATCH_SIZE, -1))
    return reduce_weighted_loss(weighted_bincrossentropy(y_true_f, y_pred_f) + dice_coef(y_true, y_pred))

## Evaluation loop
Let's define evaluation process over specified dataset

In [ ]:
def evaluate_model(prediction_function: Callable[[tf.Tensor], tf.Tensor],
                   eval_dataset: tf.data.Dataset) -> Tuple[float, float, float, float]:
    """
    Loads dataset according to file pattern and evaluates model's predictions on it.
    
    Parameters:
        model (Callable[[tf.Tensor], tf.Tensor]): Function for model inference.
        eval_dataset (tf.dataDataset): Dataset for evaluation.
    
    Returns:
        Tuple[float, float, float, float]: IoU score, recall score, precision score and mean loss.
    """
    IoU_measures = []
    recall_measures = []
    precision_measures = []
    losses = []
    
    for inputs, labels in tqdm(eval_dataset):
        # Prediction shape (N, W, H)
        predictions = prediction_function(inputs)
        for i in range(inputs.shape[0]):
            IoU_measures.append(IoU_metric(labels[i, :, :,  0], predictions[i, :, :]))
            recall_measures.append(recall_metric(labels[i, :, :,  0], predictions[i, :, :]))
            precision_measures.append(precision_metric(labels[i, :, :,  0], predictions[i, :, :]))
        labels_cleared = tf.where(labels < 0, 0, labels)
        losses.append(bce_dice_loss(labels_cleared, tf.expand_dims(tf.cast(predictions, tf.float32), axis=-1)))
            
    mean_IoU = np.mean(IoU_measures)
    mean_recall = np.mean(recall_measures)
    mean_precision = np.mean(precision_measures)
    mean_loss = np.mean(losses)
    return mean_IoU, mean_recall, mean_precision, mean_loss

# Modeling technique
Description of technique

## Model

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

def build_CNN_AE_model() -> Model:
    """
    Create CNN auto encode model.
    
    Returns:
        (Model): Keras model.
    """
    base_model = tf.keras.applications.MobileNetV2(input_shape=[32, 32, 12], include_top=False, weights=None)

    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',   # 32x32
        'block_3_expand_relu',   # 16x16
        'block_6_expand_relu',   # 8x8
        'block_13_expand_relu',  # 4x4
        'block_16_project',      # 2x2
    ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

    # Create the feature extraction model
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

    down_stack.trainable = True

    up_stack = [
        pix2pix.upsample(512, 3),  # 2x2 -> 4x4
        pix2pix.upsample(256, 3),  # 4x4 -> 8x8
        pix2pix.upsample(128, 3),  # 8x8 -> 16x16
        pix2pix.upsample(64, 3),   # 16x16 -> 32x32
    ]

    inputs = tf.keras.layers.Input(shape=[32, 32, 12])

    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
        filters=1, kernel_size=3, strides=2,
    padding='same')  #64x64 -> 128x128

    x = last(x)
    outputs = Conv2D(1, 1, padding='same', activation='sigmoid')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)
    
# Create the segmentation model
segmentation_model = build_CNN_AE_model()
segmentation_model.summary()

## Training loop

In [ ]:
def train_model(model: Model, train_dataset: tf.data.Dataset, epochs:int=10) -> Tuple[List[float], List[float]]:
    """
    Trains a model using train dataset. (Save weights of model with best IoU)
    
    Args:
        model (Model): Model to train.
        train_dataset (Dataset): Training dataset.
        epochs (int): Number of epochs
    Returns:
        Tuple[List[float], List[float]]: Train losses and Validation losses
    """
    loss_fn = bce_dice_loss
    optimizer = tf.keras.optimizers.Adam()
    batch_losses = []
    val_losses = []
    best_IoU = 0.0
    
    for epoch in range(epochs):
        losses = []
        print(f'Epoch {epoch+1}/{epochs}')
        # Iterate through the dataset
        progress = tqdm(train_dataset)
        for images, masks in progress:
            with tf.GradientTape() as tape:
                # Forward pass
                predictions = model(images, training=True)
                label = tf.where(masks < 0, 0, masks)
                # Compute the loss
                loss = loss_fn(label, predictions)
                losses.append(loss.numpy())
                progress.set_postfix({'batch_loss': loss.numpy()})
            # Compute gradients
            gradients = tape.gradient(loss, model.trainable_variables)
            # Update the model's weights
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Evaluate model
        print("Evaluation...")
        IoU, recall, precision, val_loss = evaluate_model(lambda x: tf.where(model.predict(x) > 0.5, 1, 0)[:,:,:,0], validation_dataset)
        print("Validation set metrics:")
        print(f"Mean IoU: {IoU}\nMean precision: {precision}\nMean recall: {recall}\nValidation loss: {val_loss}\n")
        # Save best model
        if IoU > best_IoU:
            best_IoU = IoU
            model.save_weights("best.h5")
        
        # Print the loss for monitoring
        print(f'Epoch: {epoch}, Train loss: {np.mean(losses)}')
        batch_losses.append(np.mean(losses))
        val_losses.append(val_loss)
    
    print(f"Best model IoU: {best_IoU}")
    return batch_losses, val_losses

# Set reproducability
tf.random.set_seed(1337)

segmentation_model = build_CNN_AE_model()
train_losses, val_losses = train_model(segmentation_model, train_dataset, epochs=15)

## Plot loss functions

In [ ]:
def plot_train_and_val_losses(train_losses, val_losses):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    axs[0].plot(train_losses)
    axs[0].set_title("train loss")
    
    axs[1].plot(val_losses)
    axs[1].set_title("validation loss")
    
    plt.show()


plot_train_and_val_losses(train_losses, val_losses)

# Testing

In [ ]:
# Load best model
segmentation_model = build_CNN_AE_model()
segmentation_model.load_weights("best.h5")

## Metrics on test set
Calculating metrics on test set

In [ ]:
def predict_on_full_image(x_batch: tf.Tensor) -> tf.Tensor:
    """
    Do model inference of full 64x64 image.
    
    Args:
        x_batch (tf.Tensor): Input batch with 64x64 images.
    Returns:
        (tf.Tensor): Prediction 64x64 image mask.
    """
    global segmentation_model
    b_size = x_batch.shape[0]
    top_left = x_batch[:, :32, :32]
    top_right = x_batch[:, :32, 32:]
    bottom_left = x_batch[:, 32:, :32]
    bottom_right = x_batch[:, 32:, 32:]

    stacked_tensor = tf.stack([top_left, top_right, bottom_left, bottom_right], axis=1)
    stacked_tensor = tf.reshape(stacked_tensor, (4*b_size, 32, 32, -1))
    result_tensor = segmentation_model.predict(stacked_tensor)

    reshaped_tensor = tf.reshape(result_tensor, (b_size, 2, 2, 32, 32, -1))
    reconstructed_tensor = tf.concat([
        tf.concat([reshaped_tensor[:, 0, 0], reshaped_tensor[:, 0, 1]], axis=2),
        tf.concat([reshaped_tensor[:, 1, 0], reshaped_tensor[:, 1, 1]], axis=2)
    ], axis=1)
    return tf.where(reconstructed_tensor[:, :, :, 0] > 0.5, 1, 0)

print("Evaluation...")
print("Test set metrics:")
IoU, recall, precision, val_loss = evaluate_model(predict_on_full_image, test_dataset)
print(f"Mean IoU: {IoU}\nMean precision: {precision}\nMean recall: {recall}\nTest loss: {val_loss}")

## Comparison with statistical model
How better is our model in comparison of prediction of only high frequency fire cells (>0.2 probability of being on fire)

In [ ]:
class NaivePredictor:
    """
    Naive predictor that predicts fire only if cell has chance being on fire more than 0.2
    """
    def __init__(self) -> None:
        """
        Initialize model and create frequency matrix
        """
        self.frequency_matrix = tf.zeros((32, 32), dtype=np.float32)
    
    def train(self, train_dataset: tf.data.Dataset) -> None:
        """
        Train by calculating frequency for each cell.
        
        Args:
            train_dataset (Dataset): Dataset to train on.
        """
        for _, labels in tqdm(train_dataset):
            label_batch = labels[:, :, :, 0]
            label_batch = tf.where(label_batch < 0, 0, label_batch)
            self.frequency_matrix = self.frequency_matrix + np.sum(label_batch, axis=0)
        self.frequency_matrix = self.frequency_matrix / np.max(self.frequency_matrix)
        self.frequency_matrix = tf.where(self.frequency_matrix > 0.2, 1, 0)
    
    def predict(self, X: tf.Tensor) -> tf.Tensor:
        """
        Dummy predict function.
        
        Args:
            train_dataset (Dataset): Dataset to train on.
        Returns:
            (Tensor): Predicted fire mask.
        """
        return tf.tile(tf.expand_dims(self.frequency_matrix, axis=0), [X.shape[0],1,1])

naive_predictor = NaivePredictor()
naive_predictor.train(train_dataset)

In [ ]:
IoU, recall, precision, val_loss = evaluate_model(naive_predictor.predict, test_dataset)
print(f"Mean IoU: {IoU}\nMean precision: {precision}\nMean recall: {recall}\nTest loss: {val_loss}")

Compare trained model with Naive one: <br/>
1. <font color="green">IoU 23% better</font>
2. <font color="green">Precision 47% better</font>
3. <font color="red">Recall 50% worse</font>

# Inference on test set

In [ ]:
def show_inference(n_rows: int, features: tf.Tensor, label: tf.Tensor, prediction_function: Callable[[tf.Tensor], tf.Tensor]) -> None:
    """
    Show model inference through images.
    
    Args:
        n_rows (int): Number of rows for subplots.
        features (tf.Tensor): Input features.
        label (tf.Tensor): True labels.
        prediction_function (Callable[[tf.Tensor], tf.Tensor]): Function for model prediction.
    """
    
    # Variables for controllong the color map for the fire masks
    CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
    BOUNDS = [-1, -0.1, 0.001, 1]
    NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
    
    fig = plt.figure(figsize=(15,n_rows*4))
    
    prediction = prediction_function(features)
    for i in range(n_rows):
        plt.subplot(n_rows, 3, i*3 + 1)
        plt.title("Previous day fire")
        plt.imshow(features[i, :, :, -1], cmap=CMAP, norm=NORM)
        plt.axis('off')
        plt.subplot(n_rows, 3, i*3 + 2)
        plt.title("True next day fire")
        plt.imshow(label[i, :, :, 0], cmap=CMAP, norm=NORM)
        plt.axis('off')
        plt.subplot(n_rows, 3, i*3 + 3)
        plt.title("Predicted next day fire")
        plt.imshow(prediction[i, :, :])
        plt.axis('off')    
    plt.tight_layout()

In [ ]:
features, labels = next(iter(test_dataset))
show_inference(5, features, labels, predict_on_full_image)